In [1]:
import yfinance as yf

ticker = 'META'
df = yf.download(ticker)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-05-18,42.049999,45.000000,38.000000,38.230000,38.151604,573576400
2012-05-21,36.529999,36.660000,33.000000,34.029999,33.960213,168192700
2012-05-22,32.610001,33.590000,30.940001,31.000000,30.936428,101786600
2012-05-23,31.370001,32.500000,31.360001,32.000000,31.934378,73600000
2012-05-24,32.950001,33.209999,31.770000,33.029999,32.962265,50237200
...,...,...,...,...,...,...
2024-08-29,519.049988,527.200012,515.679993,518.219971,518.219971,8317400
2024-08-30,521.349976,523.539978,515.200012,521.309998,521.309998,9157500
2024-09-03,519.640015,525.489990,508.619995,511.760010,511.760010,12431000


In [3]:
df = df.loc['2020-01-01':].copy()

In [5]:
df['change_tomorrow'] = df['Adj Close'].pct_change(-1)
df.change_tomorrow = df.change_tomorrow * -1
df.change_tomorrow = df.change_tomorrow * 100

In [7]:
df = df.dropna().copy()
df

,Open,High,Low,Close,Adj Close,Volume,change_tomorrow
Date,,,,,,,
2020-01-02,206.750000,209.789993,206.270004,209.779999,209.349792,12077100,-0.531935
2020-01-03,207.210007,210.399994,206.949997,208.669998,208.242081,11188400,1.848540
2020-01-06,206.699997,212.779999,206.520004,212.600006,212.164017,17058900,0.215896
2020-01-07,212.820007,214.580002,211.750000,213.059998,212.623062,14912400,1.003628
2020-01-08,213.000000,216.240005,212.610001,215.220001,214.778641,13475000,1.410910
...,...,...,...,...,...,...,...
2024-08-28,517.669983,521.090027,512.450012,516.780029,516.780029,9106100,0.277863
2024-08-29,519.049988,527.200012,515.679993,518.219971,518.219971,8317400,0.592743
2024-08-30,521.349976,523.539978,515.200012,521.309998,521.309998,9157500,-1.866107


In [9]:
y = df.change_tomorrow
X = df.drop(columns='change_tomorrow')

In [11]:
n_days = len(df.index)
n_days

1176

In [13]:
n_days_split = int(n_days*0.7)
n_days_split

823

In [15]:
X_train, y_train = X.iloc[:n_days_split], y.iloc[:n_days_split]
X_test, y_test = X.iloc[n_days_split:], y.iloc[n_days_split:]

In [17]:
from sklearn.tree import DecisionTreeRegressor

In [19]:
model_dt_split = DecisionTreeRegressor(max_depth=15, random_state=42)

In [21]:
model_dt_split.fit(X=X_train, y=y_train)

DecisionTreeRegressor(max_depth=15, random_state=42)

In [23]:
from sklearn.metrics import mean_squared_error

y_pred_test = model_dt_split.predict(X=X_test)
mean_squared_error(y_true=y_test, y_pred=y_pred_test)

4.9400964010406065

In [25]:
y_pred_train = model_dt_split.predict(X=X_train)
mean_squared_error(y_true=y_train, y_pred=y_pred_train)

3.303900331625947

In [27]:
from backtesting import Backtest, Strategy

C:\Users\Dell\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [29]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = DecisionTreeRegressor(max_depth=15, random_state=42)
        self.already_bought = False
        
        self.model.fit(X=X_train, y=y_train)

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [31]:
bt_test = Backtest(X_test, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

In [33]:
results = bt_test.run(limit_buy=1, limit_sell=-5)

df_results_test = results.to_frame(name='Values').loc[:'Return [%]']\
    .rename({'Values':'Out of Sample (Test)'}, axis=1)
df_results_test

,Out of Sample (Test)
Start,2023-04-11 00:00:00
End,2024-09-04 00:00:00
Duration,512 days 00:00:00
Exposure Time [%],96.31728
Equity Final [$],20929.861706
Equity Peak [$],22317.300305
Return [%],109.298617


In [35]:
bt_train = Backtest(X_train, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

results = bt_train.run(limit_buy=1, limit_sell=-5)

df_results_train = results.to_frame(name='Values').loc[:'Return [%]']\
    .rename({'Values':'In Sample (Train)'}, axis=1)
df_results_train

,In Sample (Train)
Start,2020-01-02 00:00:00
End,2023-04-10 00:00:00
Duration,1194 days 00:00:00
Exposure Time [%],97.326853
Equity Final [$],45836.147728
Equity Peak [$],46132.217598
Return [%],358.361477


In [37]:
import pandas as pd

In [39]:
df_results = pd.concat([df_results_train, df_results_test], axis=1)
df_results

,In Sample (Train),Out of Sample (Test)
Start,2020-01-02 00:00:00,2023-04-11 00:00:00
End,2023-04-10 00:00:00,2024-09-04 00:00:00
Duration,1194 days 00:00:00,512 days 00:00:00
Exposure Time [%],97.326853,96.31728
Equity Final [$],45836.147728,20929.861706
Equity Peak [$],46132.217598,22317.300305
Return [%],358.361477,109.298617


In [43]:
bt_test.plot(filename='reports_backtesting/regression_test_set.html')
bt_train.plot(filename='reports_backtesting/regression_train_set.html')

C:\Users\Dell\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Dell\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Dell\anaconda3\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
C:\Users\Dell\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Dell\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for '

GridPlot(id='p1969', ...)